In [12]:
import re
import string

import contractions
import gensim
import nltk
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [13]:
# word2vec
path = get_tmpfile("word2vec.model")
model = Word2Vec(common_texts,
                 size=256,
                 window=5,
                 min_count=1,
                 workers=16,
                 sg=1)
model.save("word2vec.model")
# # conti training
# model = Word2Vec.load("word2vec.model")
# model.train([["hello", "world"]], total_examples=1, epochs=1)

In [15]:
transcripts = pd.read_csv('data/transcripts.csv')['transcript']

In [17]:
def rm_num(input_str):
    return re.sub(r'\d+', '', input_str)

transcripts = transcripts.apply(contractions.fix) # remove contractions
transcripts = transcripts.str.lower() # lowercase
transcripts = transcripts.apply(rm_num) # remove numbers
specials = '()'
transcripts = transcripts.str.translate(str.maketrans('()',' '*len(specials))) # remove par around laughter
transcripts = transcripts.str.translate(str.maketrans('','',string.punctuation)) # remove punctuation
transscipts = transcripts.str.strip()

In [18]:
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# transcripts[0]

In [298]:
# model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model = KeyedVectors.load_word2vec_format('~/weight/GoogleNews-vectors-negative300.bin', binary=True)
# model.wv['computer']

In [19]:
# ! nltk.download('punkt')
stemmer = PorterStemmer()
vec_embeddings = {}
for i, transcript in enumerate(transcripts):
    vec_embeddings[i] = [stemmer.stem(word) for word in word_tokenize(transcript) if not word in stop_words]

In [20]:
vec_embeddings[0]

['good',
 'morn',
 'laughter',
 'great',
 'blown',
 'away',
 'whole',
 'thing',
 'fact',
 'leav',
 'laughter',
 'three',
 'theme',
 'run',
 'confer',
 'relev',
 'want',
 'talk',
 'one',
 'extraordinari',
 'evid',
 'human',
 'creativ',
 'present',
 'peopl',
 'varieti',
 'rang',
 'second',
 'put',
 'us',
 'place',
 'idea',
 'go',
 'happen',
 'term',
 'futur',
 'idea',
 'may',
 'play',
 'outi',
 'interest',
 'educ',
 'actual',
 'find',
 'everybodi',
 'interest',
 'educ',
 'find',
 'interest',
 'dinner',
 'parti',
 'say',
 'work',
 'educ',
 '—',
 'actual',
 'often',
 'dinner',
 'parti',
 'frankli',
 'laughter',
 'work',
 'educ',
 'ask',
 'laughter',
 'never',
 'ask',
 'back',
 'curious',
 'strang',
 'say',
 'somebodi',
 'know',
 'say',
 'say',
 'work',
 'educ',
 'see',
 'blood',
 'run',
 'face',
 'like',
 'oh',
 'god',
 'know',
 'laughter',
 'one',
 'night',
 'week',
 'laughter',
 'ask',
 'educ',
 'pin',
 'wall',
 'one',
 'thing',
 'goe',
 'deep',
 'peopl',
 'right',
 'like',
 'religion',
